In [1]:
%pip install pillow

Note: you may need to restart the kernel to use updated packages.


In [15]:
import os
from PIL import Image


def create_compact_tileset(directory, output_file):
    # Get a list of all PNG files in the directory
    files = [f for f in os.listdir(directory) if f.endswith('.png')]
    if not files:
        print("No PNG files found in the directory.")
        return

    # Open all images and sort them by area (width * height)
    images = [Image.open(os.path.join(directory, f)) for f in files]
    images.sort(key=lambda img: img.width * img.height, reverse=True)

    # Function to place images in the tileset
    def try_to_place_images(tileset_width, tileset_height):
        tileset = Image.new(
            'RGBA', (tileset_width, tileset_height), (0, 0, 0, 0))
        x, y, row_height = 0, 0, 0
        for img in images:
            if x + img.width > tileset.width:
                x = 0
                y += row_height
                row_height = 0
            if y + img.height > tileset.height:
                return None
            tileset.paste(img, (x, y))
            x += img.width
            row_height = max(row_height, img.height)
        return tileset.crop(tileset.getbbox())

    # Start with an estimated size and increase if needed
    total_area = sum(img.width * img.height for img in images)
    tileset_side = int((total_area ** 0.5) + 0.5)
    max_attempts = 100
    attempt = 0
    increment_size = max(max(img.width for img in images),
                         max(img.height for img in images))
    tileset = None

    while tileset is None and attempt < max_attempts:
        print(
            f"Attempt {attempt + 1}: Trying tileset size: {tileset_side}x{tileset_side}")
        tileset = try_to_place_images(tileset_side, tileset_side)
        if tileset is None:
            tileset_side += increment_size
        attempt += 1

    if tileset is None:
        raise ValueError(
            "Failed to create a compact tileset with the given images after maximum attempts.")

    # Save the tileset to a file
    tileset.save(output_file)
    print(f"Tileset saved as {output_file}")


# Example usage
directory = "/Users/aramhammoudeh/Documents/ThroneGameAssets/ThroneGame_HomePngs"
output_file = "output/home_tileset_compact.png"

create_compact_tileset(directory, output_file)

Attempt 1: Trying tileset size: 237x237
Attempt 2: Trying tileset size: 429x429
Tileset saved as output/home_tileset_compact.png
